In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/home/branch/rnd/research-pt3/lavender


/home/branch/miniconda3/envs/nlp/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
os.environ["TOKENIZERS_PARALLELISM"] = "true"

import json
import dotsi

import numpy as np
import pandas as pd

import torch

from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [6]:
def generate(text, model, tokenizer, device, do_sample, max_prompt_length=50, top_k=100, epsilon_cutoff=.00005, temperature=1):
    text = [tokenizer.eos_token + i + tokenizer.eos_token for i in text]
    batch = tokenizer(text, padding=True, return_tensors="pt")

    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)

    generated_ids = model.generate(input_ids=input_ids,attention_mask=attention_mask, max_new_tokens=max_prompt_length, do_sample=do_sample,top_k=top_k, epsilon_cutoff=epsilon_cutoff, temperature=temperature)
    preds = tokenizer.batch_decode(generated_ids[:, -(generated_ids.shape[1] - input_ids.shape[1]):], skip_special_tokens=True)

    return preds

In [5]:
# Arguments
device = "cpu"
modelroot = "../lavender-train/checkpoints/lave-v0.3-medium"

In [7]:
# Load config
config = dotsi.Dict(json.load(open(os.path.join(modelroot, "config.json"))))

# Load trained weights
weights_path = os.path.join(modelroot, "model-v4.ckpt")
weights = torch.load(weights_path, map_location="cpu")["state_dict"]
weights = {k.replace("model.", ""): weights[k] for k in weights.keys()}

In [8]:
# Initiate model
tokenizer = GPT2Tokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(config.model_name)
_ = model.load_state_dict(weights)
_ = model.eval()
_ = model.to(device)

In [9]:
# Generate
text = ["[concepts] fear [objects] man, woman, snake"]
generate(text, model, tokenizer, device, do_sample=True, max_prompt_length=150)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


['A man in a red cloak stands beside a woman in black who is turning away. A snake is coiling in the midground next to the woman.']